## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_34.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_34.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [6]:
Total_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [7]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [8]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

optuna

In [9]:
# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        'num_leaves': trial.suggest_int('num_leaves', 1000, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.00005, 0.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 200.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 200.0),

        'random_state': 42,
        'n_jobs': -1,
        'metric': 'auc',
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(Total_X_train, Total_y_train)
    
    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]
    
    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=800)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-17 10:17:46,292] A new study created in memory with name: no-name-5be4d927-7c92-4799-9bc0-76939a745b15
[I 2025-02-17 10:17:50,704] Trial 0 finished with value: 0.7377779055665643 and parameters: {'n_estimators': 2757, 'num_leaves': 2862, 'learning_rate': 0.1004150391695001, 'min_child_samples': 111, 'reg_alpha': 107.94115436339777, 'reg_lambda': 39.84834111915578}. Best is trial 0 with value: 0.7377779055665643.
[I 2025-02-17 10:17:55,911] Trial 1 finished with value: 0.7369836798377318 and parameters: {'n_estimators': 3617, 'num_leaves': 3260, 'learning_rate': 0.3556553756517089, 'min_child_samples': 395, 'reg_alpha': 147.83398989998287, 'reg_lambda': 108.30345387323115}. Best is trial 0 with value: 0.7377779055665643.
[I 2025-02-17 10:18:10,936] Trial 2 finished with value: 0.738603272017152 and parameters: {'n_estimators': 4200, 'num_leaves': 1731, 'learning_rate': 0.008325829158795863, 'min_child_samples': 429, 'reg_alpha': 66.58978128838662, 'reg_lambda': 58.13333997101

Best trial:
  Value: 0.7394789697314215
  Params: 
    n_estimators: 2906
    num_leaves: 2680
    learning_rate: 0.17889831086132366
    min_child_samples: 49
    reg_alpha: 48.2005369742791
    reg_lambda: 134.06515231239723


.